## 4.10 Code Exercise 4

Max Ryoo (hr2ee)

#### Set Up

Configs

In [1]:
OHCO = ['book_id', 'chap_num', 'para_num', 'sent_num', 'token_num']
epub_dir = 'epubs'

Imports

In [2]:
#pip install nltk

In [3]:
import pandas as pd
import numpy as np
from glob import glob
import re
import nltk

In [4]:
%matplotlib inline

In [5]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('tagsets')

[nltk_data] Downloading package punkt to /Users/maxryoo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/maxryoo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/maxryoo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package tagsets to /Users/maxryoo/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


True

#### Inspecting

We will be looking at the three books
- Middlemarch http://www.gutenberg.org/files/145/145-0.txt
- The Mill on the Floss http://www.gutenberg.org/files/6688/6688-0.txt
- Adam Bede http://www.gutenberg.org/files/507/507-0.txt

In [6]:
roman = '[IVXLCM]+'
caps = "[A-Z';, -]+"
chap_pats = {
    145: {
        'start_line': 23,
        'end_line': 33311,
        'volume': re.compile('^\s*BOOK\s+{}\s*$'.format(roman)),
        'chapter': re.compile('^\s*CHAPTER\s+{}\.s*$'.format(roman))
    },
    6688: {
        'start_line': 24,
        'end_line': 21270,
        # 'volume': re.compile('^\s*BOOK\s+{}\s*$'.format(roman)),
        'chapter': re.compile('^\s*Chapter\s+{}\.s*$'.format(roman))
    },
    507: {
        'start_line': 24,
        'end_line': 20702,
        # 'volume': re.compile('^\s*BOOK\s+{}\s*$'.format(roman)),
        'chapter': re.compile('^\s*Chapter\s+{}\s*$'.format(roman))
    },
}

In [7]:
def acquire_epubs(epub_list, chap_pats, OHCO=OHCO):
    
    my_lib = []
    my_doc = []

    for epub_file in epub_list:
        
        # Get PG ID from filename
        # book_id = int(epub_file.split('-')[-1].split('.')[0].replace('pg',''))
        book_id = int(epub_file.split('/')[1].split('-')[0])
        print("BOOK ID", book_id)
        
        # Import file as lines
        lines = open(epub_file, 'r', encoding='utf-8-sig').readlines()
        df = pd.DataFrame(lines, columns=['line_str'])
        df.index.name = 'line_num'
        df.line_str = df.line_str.str.strip()
        df['book_id'] = book_id
        
        # FIX CHARACTERS TO IMPROVE TOKENIZATION
        df.line_str = df.line_str.str.replace('—', ' — ')
        df.line_str = df.line_str.str.replace('-', ' - ')
        
        # Get book title and put into LIB table -- note problems, though
        book_title = re.sub(r"The Project Gutenberg eBook( of|,) ", "", df.loc[0].line_str, flags=re.IGNORECASE)
        book_title = re.sub(r"Project Gutenberg's ", "", book_title, flags=re.IGNORECASE)
        
        # Remove cruft
        a = chap_pats[book_id]['start_line'] - 1
        b = chap_pats[book_id]['end_line'] + 1
        df = df.iloc[a:b]
        
        # Chunk by chapter
        chap_lines = df.line_str.str.match(chap_pats[book_id]['chapter'])
        chap_nums = [i+1 for i in range(df.loc[chap_lines].shape[0])]
        df.loc[chap_lines, 'chap_num'] = chap_nums
        df.chap_num = df.chap_num.ffill()

        # Clean up
        df = df[~df.chap_num.isna()] # Remove chapter heading lines
        df = df.loc[~chap_lines] # Remove everything before Chapter 1
        df['chap_num'] = df['chap_num'].astype('int')
        # Group -- Note that we exclude the book level in the OHCO at this point
        df = df.groupby(OHCO[1:2]).line_str.apply(lambda x: '\n'.join(x)).to_frame() # Make big string
        
        # Split into paragrpahs
        df = df['line_str'].str.split(r'\n\n+', expand=True).stack().to_frame().rename(columns={0:'para_str'})
        df.index.names = OHCO[1:3] # MAY NOT BE NECESSARY UNTIL THE END
        df['para_str'] = df['para_str'].str.replace(r'\n', ' ').str.strip()
        df = df[~df['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs
        
        # Set index
        df['book_id'] = book_id
        df = df.reset_index().set_index(OHCO[:3])

        # Register
        my_lib.append((book_id, book_title, epub_file))
        my_doc.append(df)

    docs = pd.concat(my_doc)
    library = pd.DataFrame(my_lib, columns=['book_id', 'book_title', 'book_file']).set_index('book_id')
    print("Done.")
    return library, docs

In [8]:
epubs = [epub for epub in sorted(glob('data/*.txt'))]
epubs

['data/145-0.txt', 'data/507-0.txt', 'data/6688-0.txt']

In [9]:
LIB, DOC = acquire_epubs(epubs, chap_pats)

/var/folders/pn/dgy7ckd90nl7mlj6g6rc_1kw0000gn/T/ipykernel_43534/4092773172.py:49: FutureWarning: The default value of regex will change from True to False in a future version.
  df['para_str'] = df['para_str'].str.replace(r'\n', ' ').str.strip()


BOOK ID 145
BOOK ID 507
BOOK ID 6688
Done.


In [10]:
LIB

,book_title,book_file
book_id,,
145,"Middlemarch, by George Eliot",data/145-0.txt
507,"Adam Bede, by George Eliot",data/507-0.txt
6688,"The Mill on the Floss, by George Eliot",data/6688-0.txt


In [11]:
DOC.sample(10)

para_str
book_id chap_num para_num                                                   
507     2        21        The stronger curiosity of the women had drawn ...
145     143      16        “Caleb likes taking trouble: he is one of thos...
        94       16                “Now, Cadwallader, has he got any heart?”
        106      27        For the first time since Dorothea had known hi...
        140      58        “I thought you would like to cherish her memor...
507     7        20        “Aunt doesn’t like me to go a - walking only w...
6688    39       36                                           “Yes, father.”
145     103      23        “Men of your profession don’t generally smoke,...
        88       25        “In pigeon - holes partly,” said Mr. Casaubon,...
507     52       22        “But if a new feeling has come into your mind,...

#### Tokenize and Annotate

In [12]:
def tokenize(doc_df, OHCO=OHCO, remove_pos_tuple=False, ws=False):
    
    # Paragraphs to Sentences
    df = doc_df.para_str\
        .apply(lambda x: pd.Series(nltk.sent_tokenize(x)))\
        .stack()\
        .to_frame()\
        .rename(columns={0:'sent_str'})
    
    # Sentences to Tokens
    # Local function to pick tokenizer
    def word_tokenize(x):
        if ws:
            s = pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x)))
        else:
            s = pd.Series(nltk.pos_tag(nltk.word_tokenize(x))) # Discards stuff in between
        return s
            
    df = df.sent_str\
        .apply(word_tokenize)\
        .stack()\
        .to_frame()\
        .rename(columns={0:'pos_tuple'})
    
    # Grab info from tuple
    df['pos'] = df.pos_tuple.apply(lambda x: x[1])
    df['token_str'] = df.pos_tuple.apply(lambda x: x[0])
    if remove_pos_tuple:
        df = df.drop('pos_tuple', 1)
    
    # Add index
    df.index.names = OHCO
    
    return df

In [13]:
%%time
TOKEN = tokenize(DOC, ws=False)

CPU times: user 37.3 s, sys: 660 ms, total: 38 s
Wall time: 38 s


In [14]:
TOKEN.head()

pos_tuple  pos token_str
book_id chap_num para_num sent_num token_num                            
145     12       0        0        0          (BOOK, NNP)  NNP      BOOK
                                   1            (II, NNP)  NNP        II
                                   2               (., .)    .         .
                          1        0           (OLD, NNP)  NNP       OLD
                                   1            (AND, CC)   CC       AND

In [15]:
TOKEN[TOKEN.pos.str.match('^CC')]

pos_tuple pos token_str
book_id chap_num para_num sent_num token_num                         
145     12       0        1        1          (AND, CC)  CC       AND
        52       0        1        2          (AND, CC)  CC       AND
        86       2        0        11         (and, CC)  CC       and
                                   69         (and, CC)  CC       and
                          1        10         (and, CC)  CC       and
...                                                 ...  ..       ...
6688    58       65       0        26         (and, CC)  CC       and
                                   43         (and, CC)  CC       and
                 66       0        14         (but, CC)  CC       but
                 68       0        7          (and, CC)  CC       and
                                   11         (and, CC)  CC       and

[29728 rows x 3 columns]

#### Reduce

Extract a vocabulary from the TOKEN table

In [16]:
TOKEN['term_str'] = TOKEN['token_str'].str.lower().str.replace('[\W_]', '')

/var/folders/pn/dgy7ckd90nl7mlj6g6rc_1kw0000gn/T/ipykernel_43534/1858674674.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  TOKEN['term_str'] = TOKEN['token_str'].str.lower().str.replace('[\W_]', '')


In [17]:
VOCAB = TOKEN.term_str.value_counts()\
    .to_frame()\
    .rename(columns={'index':'term_str', 'term_str':'n'})\
    .sort_index()\
    .reset_index()\
    .rename(columns={'index':'term_str'})
VOCAB.index.name = 'term_id'

In [18]:
VOCAB['num'] = VOCAB.term_str.str.match("\d+").astype('int')

In [19]:
VOCAB.head()

,term_str,n,num
term_id,,,
0,,145933,0
1,1,1,1
2,1790,1,1
3,1799,2,1
4,1801,1,1


#### Annotate (VOCAB)

Add Stopwords

In [20]:
sw = pd.DataFrame(nltk.corpus.stopwords.words('english'), columns=['term_str'])
sw = sw.reset_index().set_index('term_str')
sw.columns = ['dummy']
sw.dummy = 1

In [21]:
sw.sample(10)

,dummy
term_str,
who,1
won't,1
ve,1
other,1
with,1
hasn,1
had,1
isn't,1
do,1


In [22]:
VOCAB['stop'] = VOCAB.term_str.map(sw.dummy)
VOCAB['stop'] = VOCAB['stop'].fillna(0).astype('int')

In [23]:
VOCAB[VOCAB.stop == 1].sample(10)

,term_str,n,num,stop
term_id,,,,
18860,the,31324,0,1
11201,m,585,0,1
12510,nor,252,0,1
416,against,543,0,1
10195,itself,182,0,1
12942,ourselves,56,0,1
21225,won,202,0,1
12739,of,18422,0,1
12048,most,470,0,1


Add Stems

In [24]:
from nltk.stem.porter import PorterStemmer
stemmer1 = PorterStemmer()
VOCAB['stem_porter'] = VOCAB.term_str.apply(stemmer1.stem)

In [25]:
VOCAB.sample(10)

,term_str,n,num,stop,stem_porter
term_id,,,,,
17094,skewers,1,0,0,skewer
21205,wittily,1,0,0,wittili
7933,gateway,2,0,0,gateway
13034,overmastering,7,0,0,overmast
4338,cromwell,2,0,0,cromwel
15851,rigor,4,0,0,rigor
12830,opera,7,0,0,opera
9173,humbly,5,0,0,humbl
18959,thorn,13,0,0,thorn


#### pos_max feature
Finally, add a feature named "pos_max" to the VOCAB table that contains the most frequently associated part-of-speech tag, as found in the TOKEN table, with each term

In [26]:
TOKEN.sample(10)

pos_tuple  pos token_str  \
book_id chap_num para_num sent_num token_num                                   
507     54       8        1        34                (as, IN)   IN        as   
145     165      11       0        10              (him, PRP)  PRP       him   
6688    30       4        13       28         (smallest, JJS)  JJS  smallest   
145     118      2        5        13                (on, IN)   IN        on   
        111      33       2        18                (to, TO)   TO        to   
        139      59       0        66                  (,, ,)    ,         ,   
        136      17       0        5                (sad, JJ)   JJ       sad   
6688    1        1        0        23          (rushing, VBG)  VBG   rushing   
145     142      24       6        38         (pressing, VBG)  VBG  pressing   
507     36       24       0        41                  (., .)    .         .   

                                              term_str  
book_id chap_num para_num sent_num token_num            
507     54       8        1        34               as  
145     165      11       0        10              him  
6688    30       4        13       28         smallest  
145     118      2        5        13               on  
        111      33       2        18               to  
        139      59       0        66                   
        136      17       0        5               sad  
6688    1        1        0        23          rushing  
145     142      24       6        38         pressing  
507     36       24       0        41

In [27]:
part_in_token = TOKEN.groupby(['pos','term_str']).size().reset_index()
part_in_token

,pos,term_str,0
0,'',,190
1,'',harriet,1
2,'',him,1
3,'',lad,1
4,'',madam,1
...,...,...,...
32326,WRB,where,719
32327,WRB,wherever,3
32328,WRB,whichever,1
32329,WRB,why,569


In [28]:
part_in_token.groupby(['pos','term_str'])[0].max().reset_index()

,pos,term_str,0
0,'',,190
1,'',harriet,1
2,'',him,1
3,'',lad,1
4,'',madam,1
...,...,...,...
32326,WRB,where,719
32327,WRB,wherever,3
32328,WRB,whichever,1
32329,WRB,why,569


In [29]:
pos_dict = part_in_token.groupby(['pos','term_str'])[0]\
        .max()\
        .reset_index()\
        .set_index('term_str')\
        .to_dict()['pos']

In [30]:
VOCAB.sample(10)

,term_str,n,num,stop,stem_porter
term_id,,,,,
3521,commodities,6,0,0,commod
4786,degraded,4,0,0,degrad
20602,virtuously,2,0,0,virtuous
17599,sphinxes,2,0,0,sphinx
21179,witches,2,0,0,witch
5656,dove,1,0,0,dove
14940,rag,6,0,0,rag
971,arithmetic,9,0,0,arithmet
16866,shouldst,1,0,0,shouldst


In [31]:
VOCAB['term_str'].map(pos_dict)

term_id
0        WRB
1         CD
2         CD
3         CD
4         CD
        ... 
21503     NN
21504    NNP
21505    NNP
21506     JJ
21507    NNP
Name: term_str, Length: 21508, dtype: object

In [32]:
VOCAB['pos_max'] = VOCAB['term_str'].map(pos_dict)

In [36]:
VOCAB

,term_str,n,num,stop,stem_porter,pos_max
term_id,,,,,,
0,,145933,0,0,,WRB
1,1,1,1,0,1,CD
2,1790,1,1,0,1790,CD
3,1799,2,1,0,1799,CD
4,1801,1,1,0,1801,CD
...,...,...,...,...,...,...
21503,œuvre,1,0,0,œuvr,NN
21504,μέγεθος,1,0,0,μέγεθος,NNP
21505,τι,1,0,0,τι,NNP


In [37]:
VOCAB.to_csv('outp)

'term_id,term_str,n,num,stop,stem_porter,pos_max\n0,,145933,0,0,,WRB\n1,1,1,1,0,1,CD\n2,1790,1,1,0,1790,CD\n3,1799,2,1,0,1799,CD\n4,1801,1,1,0,1801,CD\n5,1807,1,1,0,1807,CD\n6,1825,1,1,0,1825,CD\n7,1826,1,1,0,1826,CD\n8,1828,1,1,0,1828,CD\n9,1829,2,1,0,1829,CD\n10,1831,1,1,0,1831,CD\n11,1832,1,1,0,1832,CD\n12,19,1,1,0,19,CD\n13,1st,6,1,0,1st,CD\n14,2,3,1,0,2,CD\n15,20th,1,1,0,20th,CD\n16,235,1,1,0,235,CD\n17,25th,1,1,0,25th,CD\n18,27th,1,1,0,27th,CD\n19,2d,8,1,0,2d,CD\n20,2r,1,1,0,2r,CD\n21,3,1,1,0,3,CD\n22,30th,3,1,0,30th,JJ\n23,5th,1,1,0,5th,CD\n24,9th,1,1,0,9th,CD\n25,a,17140,0,1,a,NNP\n26,aaron,1,0,0,aaron,NNP\n27,abandon,4,0,0,abandon,VB\n28,abandoned,2,0,0,abandon,VBN\n29,abandoning,4,0,0,abandon,VBG\n30,abandonment,4,0,0,abandon,NN\n31,abasement,2,0,0,abas,NN\n32,abate,1,0,0,abat,VB\n33,abated,3,0,0,abat,VBN\n34,abbey,8,0,0,abbey,NNP\n35,abbot,2,0,0,abbot,NNP\n36,abbott,2,0,0,abbott,NNP\n37,abbreviate,1,0,0,abbrevi,VB\n38,abbreviated,2,0,0,abbrevi,JJ\n39,abdicated,1,0,0,abdic,VB

In [ ]:
pd.to_csv(TOKEN)

In [ ]:
pd.to_csv(LIB)

In [ ]:
pd.to_csv(DOC)